# Tarea 02 - Análisis de datos geoespaciales mediante Fiona y Shapely

José P. Barrantes - B0043 <br>
Ricardo Corrales Barquero - B32090

En el código a continuación se pretende adquirir datos disponibles en línea, y procesarlos para generar un archivo que contiene información de los límites cantonales, así como de la densidad de carreteras en cada cantón.

### Librerías

In [1]:
from owslib.wfs import WebFeatureService # obtención del WFS en línea
from geojson import dump                 # exportación de .geojson
import requests                          # herramienta para solicitudes HTTP
import fiona                             # herramienta I/O para datos GIS
from shapely.geometry import shape, mapping # geometrias
import fiona.transform                   # reproyecciones
from fiona import crs                    # sistemas de cordenadas y funciones
from tqdm import tqdm                    # barra de progreso para la generación de .gpkg

## Obtención de datos
En el [sitio web del SNIT](https://www.snitcr.go.cr/) podemos obtener la url de los WFS que necesitaremos. Una vez las tenemos, podemos consultar las capas que vamos a utilizar en esta pŕactica.

In [2]:
wfs_cartografia_1_5mil = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_5/wfs?', version='1.1.0')
list(wfs_cartografia_1_5mil.contents)  # para visualizar el contenido

['IGN_5:forestal2017_5k',
 'IGN_5:indice_5000',
 'IGN_5:cultivos2017_5k',
 'IGN_5:curvas_5000',
 'IGN_5:delimitacion2017_5k',
 'IGN_5:edificaciones2017_5k',
 'IGN_5:hidrografia_5000',
 'IGN_5:limitecantonal_5k',
 'IGN_5:limitedistrital_5k',
 'IGN_5:limiteprovincial_5k',
 'IGN_5:linea_costa_5000',
 'IGN_5:pastos2017_5k',
 'IGN_5:urbano_5000',
 'IGN_5:vias_5000']

De acá, la capa 'IGN_5:limitecantonal_5k' es la que nos resulta de interés. Procedemos a descargar la capa con las divisiones cantonales.

In [3]:
# Solicitud de capa WFS de límite cantonal mediante GET, para retornarse como JSON

# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_5:limitecantonal_5k',   # la de interés
              srsName='urn:ogc:def:crs:EPSG::4326',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_5/wfs?", params=params)

# Descarga de la respuesta en un archivo GeoJSON

with open('./datos/limite_cantonal.geojson', 'w') as file:
    dump(response.json(), file)

Guardamos esta capa en el archivo 'limite_cantonal.geojson'

Hacemos lo mismo para obtener la información de las carreteras.

In [4]:
wfs_cartografia_1_200mil = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_200/wfs?version=1.1.0', version='1.1.0')
list(wfs_cartografia_1_200mil.contents)

['IGN_200:aeropuertointernacional_200k',
 'IGN_200:aerodromos_200k',
 'IGN_200:bordecostarica_200k',
 'IGN_200:cotafotogrametrica_200k',
 'IGN_200:curvas_de_nivel_200k',
 'IGN_200:edificaciones_y_construcciones_200k',
 'IGN_200:embalses_200k',
 'IGN_200:estacionferroviaria_200k',
 'IGN_200:hojas_200_completas',
 'IGN_200:lago_o_laguna_200k',
 'IGN_200:Laguna_Intermitente_1_200mil',
 'IGN_200:lineas_de_costa_200k',
 'IGN_200:muelle_200k',
 'IGN_200:redvial_200k',
 'IGN_200:reddrenaje_200k',
 'IGN_200:salinas_200k',
 'IGN_200:viaferrea_200k']

In [5]:
# Solicitud de capa WFS de red vial mediante GET, para retornarse como JSON

# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_200:redvial_200k',       # la de interés
              srsName='urn:ogc:def:crs:EPSG::4326',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_200/wfs?version=1.1.0", params=params)

# Descarga de la respuesta en un archivo GeoJSON

with open('./datos/red_vial.geojson', 'w') as file:
    dump(response.json(), file)

La capa de las carreteras la guardamos como 'red_vial.geojson'

## Lectura de los datos con fiona

In [6]:
cantones = fiona.open('./datos/limite_cantonal.geojson')

## Intersección

El siguiente código calcula la desidad de carreteras para cada cantón, y genera un archivo .gpkg que contiene la información.

In [7]:
schema_final = {'geometry': 'Unknown',
                'properties': {'total_long_vias': 'float',
                              'area': 'float',
                              'densidad_vial': 'float'}}

with fiona.collection('./datos/densidad_vial.gpkg',
                      mode='w',
                      schema=schema_final,
                      driver='GPKG',
                      crs=fiona.crs.from_epsg(4326),
                      layer='densidad-vial') as collection, \
    fiona.open('./datos/limite_cantonal.geojson') as cantones, \
    fiona.open('./datos/red_vial.geojson') as vias:
    for canton in tqdm(cantones):
        total_long_vias = 0
        # Se transforma la geometría al EPSG oficial de CR para medir en metros.
        geometria_canton = shape(fiona.transform.transform_geom('EPSG:4326', 'EPSG:5367', canton['geometry']))
        for via in vias:
            # Esto calcula, para cada cantón y cada vía del país, la longitud de su intersección.
            geometria_calle = shape(fiona.transform.transform_geom('EPSG:4326', 'EPSG:5367', via['geometry']))
            total_long_vias += geometria_canton.intersection(geometria_calle).length / 1000 # dividimos entre 1000 para pasar de m a km
        area = geometria_canton.area / 1000000 # viene en m^2, lo ajustamos a km^2
        densidad_vial = total_long_vias / area
        collection.write({
            'geometry': canton['geometry'],
            'properties': {'total_long_vias': total_long_vias,
                          'area': area,
                          'densidad_vial': densidad_vial}
        })
                

100%|██████████| 82/82 [12:21<00:00,  9.04s/it]
